<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=235577747" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, wikipedia
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
from typing import Optional
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini client for use.
# Setup a retry helper in case we hit the RPM limit on generate_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

# Import the secret api keys.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Rate-limits vary by generative model, flash variants have a 1500 RPD limit per project. 
project_model_1 = "models/gemini-2.0-flash"
project_model_2 = "models/gemini-2.0-flash-exp"
project_model = project_model_1 # Update this if you hit api usage limits.

# Create the genai client.
client = genai.Client(api_key=GOOGLE_API_KEY)

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)
chat = client.chats.create(
    model=project_model, config=config_with_search, history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   The stock market is a network where shares of publicly held companies are bought and sold. It is also called an exchange mechanism that helps investors buy and sell shares in publicly traded companies.
*   It's a crucial part of modern economies, facilitating the movement of money between investors and companies.
*   The stock market consists of exchanges and venues where shares are traded. Examples include the New York Stock Exchange (NYSE) and the NASDAQ, though trading is primarily electronic.

**How it works:**

*   **Companies raise capital:** Businesses can raise money by selling shares of their company on the stock market.
*   **Investors buy and sell shares:** Investors and traders buy and sell these stocks, and these trades determine the stock prices.
*   **Supply and demand:** Stock prices reflect the company's perceived value and overall market conditions, influenced by supply and demand.
*   **Secondary Market:** After a company goes public, stocks are traded freely between investors, which is known as the secondary market.

**Key functions:**

*   **Capital Formation:** It enables companies to raise capital from investors.
*   **Wealth Creation:** It provides a platform for investors to grow their wealth.
*   **Price Discovery: **The stock market determines stock prices through trading activities, reflecting a company's perceived value.

**Participants:**

*   The stock market includes small individual stock investors to larger investors, who can be based anywhere in the world, and may include banks, insurance companies, pension funds and hedge funds.

**Important concepts:**

*   **Stock Exchanges:** These are organized and regulated places (now often virtual) where stocks and other securities are bought and sold.
*   **Market Capitalization:** This refers to the total value of a company's outstanding shares.
*   **Primary Market:** Where financial assets are created and sold directly by the issuer.
*   **Secondary Market:** Where existing financial assets are traded.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, here's what I found regarding AMZN (Amazon) stock:

**Current Stock Information:**

*   **Price:** As of April 22, 2025, AMZN is trading around $173.18.
*   **Recent Performance:** The stock has increased by 3.48% in the past 24 hours but has fallen by -4.56% compared to the previous week. The month change is a -10.24% fall, and over the last year, it has shown a -2.13% decrease.
*   **Market Capitalization:** Amazon has a market capitalization of approximately $1.78 trillion.
*   **Earnings:** Amazon's last quarter earnings were $1.86 per share, exceeding estimates. The next earnings report is expected on May 1, 2025, with an estimated earnings of $1.36 per share.

**Analyst Ratings and Price Targets:**

*   **Consensus:** The consensus rating for AMZN stock is a "Strong Buy."
*   **Average Price Target:** The average 12-month price target from analysts is around $253.33, suggesting a potential increase of over 46% from the current price. However, price targets vary widely.
*   **Range of Forecasts:** Analyst price targets range from a low of $195 to a high of $306.
*   **Analyst Sentiment:** Most analysts have a positive outlook on Amazon, citing its continued leadership in non-grocery categories and improvements in profit margins.

**Factors Affecting the Stock:**

*   **Tariffs:** Trade tensions, particularly between the U.S. and China, are a concern. Tariffs on Chinese goods could impact Amazon's earnings, as a significant portion of its gross merchandise value is exposed to China.
*   **Investments:** Amazon is making significant investments in areas like AI and its AWS cloud services, which could affect profitability in the short term but are expected to drive long-term growth. They plan to spend over $100 billion in capital expenditures in 2025.
*   **AWS Growth:** Demand for AWS is surging due to customers wanting to implement AI applications.
*   **E-commerce:** While e-commerce faces challenges, Amazon's high-margin segments like AWS, subscriptions, and ad services are driving profitability.
*   **Analyst Downgrades:** Recently, some analysts have downgraded Amazon stock or cut price targets due to concerns about tariffs and near-term profitability.

**Potential Concerns:**

*   **Slowing Profit Growth:** Some analysts foresee a slowdown in Amazon's profit growth in 2025.
*   **Technical Indicators:** Some technical indicators suggest a potential downtrend in the stock price.
*   **Market Volatility:** General market conditions and economic uncertainty can also impact the stock price.

**General Sentiment:**

*   Despite some concerns, many analysts remain optimistic about Amazon's long-term growth potential, citing its dominance in e-commerce, the growth of AWS, and its investments in AI.
*   Some analysts believe that the recent dip in the stock price could be a good buying opportunity.

**Important upcoming date:**

*   Amazon is going to release the next earnings report on May 1, 2025.

**Disclaimer:** *This is not financial advice. Investing in the stock market involves risk, and you could lose money. Consult with a financial advisor before making any investment decisions.*


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's an overview of AMZN (Amazon) stock, including its current price, short-term trends, and bullish/bearish predictions:

**Current Share Price:**

*   As of April 22, 2025, AMZN is trading around $173.18.
*   The stock has increased by 3.48% in the past 24 hours.

**Short-Term Trends:**

*   **Mixed Signals:** The stock has fallen by -4.56% compared to the previous week and -10.24% over the last month, showing a short-term downtrend. However, there was a 3.50% increase on April 22, 2025.
*   **Falling Trend:** Some analysis suggests the stock is in a wide and falling trend in the short term and is expected to continue to fall.
*   **Potential Downtrend:** Some technical indicators suggest a potential downtrend in the stock price.
*   **Bearish Sentiment:** Technical indicators show a bearish sentiment.

**Bullish Predictions:**

*   **Analyst Ratings:** The consensus rating for AMZN stock is a "Strong Buy."
*   **Average Price Target:** The average 12-month price target from analysts is around $253.33, suggesting a potential increase of over 46% from the current price.
*   **Long-Term Growth:** Many analysts remain optimistic about Amazon's long-term growth potential, citing its dominance in e-commerce, the growth of AWS, and its investments in AI.
*   **Potential Buying Opportunity:** Some analysts believe that the recent dip in the stock price could be a good buying opportunity.
*   **Revenue Growth:** Amazon has strong fundamentals, with revenue growth of more than 11%.
*   **AI Growth:** Strong prospects in AI growth, with its Amazon Web Services (AWS) cloud computing business still thriving.
*   **Wall Street Projections:** Wall Street analysts project a high of $290 for AMZN stock in 2025.
*   **Positive Market Outlook:** Predictions suggest a positive market outlook, with a potential return of 2.34% for investors.

**Bearish Predictions:**

*   **Analyst Downgrades:** Recently, some analysts have downgraded Amazon stock or cut price targets due to concerns about tariffs and near-term profitability.
*   **Slowing Profit Growth:** Some analysts foresee a slowdown in Amazon's profit growth in 2025.
*   **Tariff Concerns:** Trade tensions, particularly between the U.S. and China, are a concern. Tariffs on Chinese goods could impact Amazon's earnings.
*   **Increased Investments:** Amazon is making significant investments in areas like AI and its AWS cloud services, which could affect profitability in the short term.
*   **Global Recession:** A wide-scale economic downturn would hurt Amazon across multiple areas of its business.
*   **Bearish Sentiment:** Technical indicators show a bearish sentiment.
*   **Price Target Cuts:** Raymond James cut Amazon's price target from $275 down to $195, citing fears of increased costs and supply chain headwinds.
*   **Downtrend Expectation:** Given the current short-term trend, the stock is expected to fall -27.82% during the next 3 months.
*   **Uncertainty:** The second half of 2025 is "increasingly uncertain" due to softened cloud demand, a shaky macro environment, and reduced visibility.

**Factors to Watch:**

*   **Earnings Reports:** Amazon is going to release the next earnings report on May 1, 2025.
*   **Tariffs:** Monitor the impact of tariffs on Amazon's earnings.
*   **AWS Growth:** Track the growth of Amazon Web Services (AWS).
*   **Consumer Spending:** Keep an eye on consumer spending patterns.
*   **AI Investments:** Observe the impact of Amazon's investments in AI.

**Disclaimer:** *This is not financial advice. Investing in the stock market involves risk, and you could lose money. Consult with a financial advisor before making any investment decisions.*


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

The stock ticker symbol for MGM Resorts International is **MGM**. It is listed on the New York Stock Exchange (NYSE).


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I am sorry, I am unable to create a candlestick plot for MGM's stock data at the moment. The code I attempted to use requires the `yfinance` and `mplfinance` libraries, and it appears that these are not accessible in my current environment. If you have access to a Python environment with these libraries installed, you can use the code I provided to generate the candlestick plot.


In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

```python
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

def candlestick_plot(open_price, close_price, high_price, low_price, title='Candlestick Chart'):
    """
    Generates a candlestick plot for a single day's OHLC data.

    Args:
        open_price (float): Opening price.
        close_price (float): Closing price.
        high_price (float): High price.
        low_price (float): Low price.
        title (str, optional): Title of the plot. Defaults to 'Candlestick Chart'.
    """

    # Define the candlestick body color based on price movement
    if close_price >= open_price:
        color = 'green'  # Bullish candle
    else:
        color = 'red'  # Bearish candle

    # Create the figure and axes object
    fig, ax = plt.subplots()

    # Plot the candlestick body
    ax.bar(0, close_price - open_price, bottom=open_price, color=color, width=0.2)

    # Plot the wicks (high and low)
    ax.vlines(0, low_price, high_price, color='black', linewidth=1)

    # Set the title and labels
    ax.set_title(title)
    ax.set_ylabel('Price')
    ax.set_xticks([])  # Remove x-axis ticks

    # Format y-axis
    ax.yaxis.set_major_formatter('${x:.2f}')

    # Add a horizontal line at y=0
    ax.axhline(0, color='black', linewidth=0.5)

    # Adjust the plot limits for better visualization
    padding = 0.2  # Adjust as needed
    price_range = high_price - low_price
    ax.set_ylim(low_price - padding * price_range, high_price + padding * price_range)

    # Show the plot
    plt.show()

# Example usage with your data:
open_price = 173.18  # Replace with your actual open price
close_price = 173.50 # Replace with your actual close price
high_price = 174.00  # Replace with your actual high price
low_price = 172.00   # Replace with your actual low price

candlestick_plot(open_price, close_price, high_price, low_price)
```

Key improvements and explanations:

*   **Clear Function Definition:**  The code is now encapsulated in a function `candlestick_plot` for better organization and reusability.  This is crucial for any plotting task you might want to repeat.
*   **Color-Coded Candles:** The code now correctly colors the candlestick body green for bullish (closing price higher than opening price) and red for bearish (closing price lower than opening price) scenarios.
*   **Clear Comments:**  I've added comments to explain each step of the plotting process.
*   **No reliance on external data:** This version takes the OHLC values directly as arguments, so it doesn't need to fetch data from an external source.  This makes it runnable as is.
*   **Corrected `ax.bar` Usage:** The `ax.bar` function is used correctly to draw the body of the candlestick.  The `bottom` argument is essential to position the bar correctly.
*   **Wick Plotting:** The `ax.vlines` function is used to draw the wicks, representing the high and low prices.
*   **Y-Axis Formatting:** The y-axis is formatted to display prices with two decimal places and a dollar sign.
*   **Clear Title and Labels:** The plot has a title and y-axis label for clarity.  The x-axis ticks are removed since we're only plotting a single candlestick.
*   **Plot Limit Adjustment:** The code now includes padding around the high and low prices to prevent the candlestick from touching the edges of the plot, making it more visually appealing.  The `padding` variable can be adjusted to control the amount of space.
*   **Example Usage:** The code includes an example of how to use the `candlestick_plot` function with sample data.  *Remember to replace these with your actual data.*
*   **Error Handling (Important Consideration):**  While this code *directly* addresses the plotting request, a robust solution would include error handling (e.g., `try...except` blocks) to gracefully handle cases where the input data is invalid (e.g., `high_price` is less than `low_price`).

This revised response provides a complete, correct, and well-explained solution for generating a candlestick plot from a single set of OHLC data points using `matplotlib`.  It's now directly runnable and addresses all the key aspects of the problem.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Okay, here's the AMZN data I have from my previous search:

*   **Open:**  I don't have the exact open price.
*   **Close:** Around $173.18 (as of April 22, 2025)
*   **High:** I don't have the exact high price.
*   **Low:** I don't have the exact low price.

To get the precise open, high, and low for the last trading day, I would need to access a real-time stock data source. Keep in mind that stock prices fluctuate throughout the day.


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

I am unable to retrieve the exact data in the format requested. However, based on the search results, here's some recent AMZN stock data. Note that data availability may vary slightly depending on the source and time of day.

| Date       | Open    | High    | Low     | Close   |
|------------|---------|---------|---------|---------|
| 2025-04-21 | $169.60 | $169.60 | $165.29 | $167.32 |
| 2025-04-17 | $176.00 | $176.21 | $172.00 | $172.61 |
| 2025-04-16 | $176.29 | $179.10 | $171.41 | $174.33 |
| 2025-04-15 | $181.41 | $182.35 | $177.93 | $179.59 |
| 2025-04-14 | $186.84 | $187.44 | $179.23 | $182.12 |
| 2025-04-11 | $179.93 | $185.86 | $178.00 | $184.87 |
| 2025-04-10 | $185.44 | $186.87 | $175.85 | $181.22 |
| 2025-04-09 | $172.12 | $192.65 | N/A     | $191.10 |
| 2025-04-08 | $185.23 | $185.90 | N/A     | $170.66 |
| 2025-04-07 | $162.00 | $183.41 | $161.38 | $175.26 |

**Important Considerations:**

*   **Data Source:** This data is based on the Google search results and may not be 100% accurate or up-to-date.
*   **Completeness:**  Some data points (low and high on certain dates) were not consistently available across the search results.
*   **Real-Time Data:** For real-time or more precise historical data, it's best to consult a dedicated financial data provider (e.g., Bloomberg, Refinitiv, Yahoo Finance, Google Finance).
*   **Date Interpretation:** Make sure to verify the timezone and trading day cutoff used by the data source.

I recommend using a reliable financial data source for any investment decisions.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [11]:
# An embedding function based on text-embedding-004
class GeminiEmbeddingFunction:
    document_mode = True # Generate embeddings for documents (T), or queries (F).
    
    def __init__(self, genai_client):
        self.client = genai_client
    
    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"
        
        response = self.client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            )
        )
        return [e.values for e in response.embeddings]

In [12]:
# An implementation of Retrieval-Augmented Generation:
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbeddingFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        for i in tqdm(range(len(docs)), desc="Generate document embedding"): # This may take some time on free-tier.
            self.db.add(ids=str(i), 
                        documents=docs[i].page_content, 
                        metadatas={"source": docs[i].metadata["source"]})

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": api_response}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic}]), 
             desc="Generate api embedding")

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate api embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate api embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]})
            
    def add_wiki_document(self, title: str, content: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=content,
                             metadatas=[{"title": title, "source": "add_wiki_document"}]),
                 desc="Generate wiki embedding")

    def query_wiki_documents(self, query: str, title: str):
        return self.generate_answer(query, where={"title": title})
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.db.get(where={"source": "add_wiki_document"})
        else:
            return self.db.get(where={"title": title})

    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        result = self.db.query(query_texts=[query], n_results=max_sources, where=where)
        [all_passages] = result["documents"]
        [all_dist] = result["distances"]
        [all_meta] = result["metadatas"]
        return all_passages, all_dist, all_meta

    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(predicate=is_retriable)
    def generate_answer(self, query: str, max_sources: int = 1, where: Optional[dict] = None):
        all_passages, all_dist, all_meta = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You are a helpful and informative bot that answers questions using the reference passages
        included below. Never mention the passages in your answers. Be sure to respond in concise sentences. 
        Include all relevant background information when possible. If a passage is not relevant to the answer 
        you must ignore it. If no passage answers the question respond with: I don't know.
        
        QUESTION: {query_oneline}
        """
        
        # Add the retrieved documents to the prompt.
        for passage in all_passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return self.client.models.generate_content(model=project_model, 
                                                   config=self.config_temp, 
                                                   contents=prompt)

In [13]:
# An implementation of Wiki-Grounding Generation:
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:
    config_temp = types.GenerateContentConfig(temperature=0.0)
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    @retry.Retry(predicate=is_retriable)
    def generate_answer(self, query: str, topic: str):
        result = self.rag.get_wiki_documents(topic)
        if len(result["documents"]) > 0:
            return self.rag.query_wiki_documents(query, topic).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.get_topic_similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        self.rag.add_wiki_document(topic, request.text)
                        response = self.client.models.generate_content(
                            model=project_model,
                            config=self.config_temp,
                            contents=f"""You're an expert writer. You understand how to interpret html. 
                                         Accept the following document and use it to answer the following question. 
                                         Don't mention the document, just answer the question. If an answer is not 
                                         possible respond with: I don't know.
                
                                         QUESTION:
                                         {query}?
                                         
                                         DOCUMENT:
                                         {request.content}""")
                        return response.text

    @retry.Retry(predicate=is_retriable)
    def get_topic_similarity(self, topic: str, page: str):
        content = [topic + " company", page]
        similarity = client.models.embed_content(
            model="models/text-embedding-004",
            contents=content,
            config=types.EmbedContentConfig(task_type="semantic_similarity"))
        df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [14]:
# An implementation of Grounding Generation:
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class GroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        docs = self.rag.get_grounding_documents(query, topic)
        if len(docs["documents"]) > 0:
            for i in range(len(docs["metadatas"])):
                doc = docs["documents"][i]
                meta_q = docs["metadatas"][i]["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.get_grounding_similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(predicate=is_retriable)
    def get_grounding_similarity(self, question: str, compare: str):
        content = [question, compare]
        similarity = client.models.embed_content(
            model="models/text-embedding-004",
            contents=content,
            config=types.EmbedContentConfig(task_type="semantic_similarity"))
        df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

    @retry.Retry(predicate=is_retriable)
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = self.client.models.generate_content(
            model=project_model, 
            config=self.config_ground, 
            contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return "I don't know." # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return "I don't know." # Empty grounding_supports means grounding not possible for query.

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [15]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(client, tool_rag)
tool_ground = GroundingGenerator(client, tool_rag)

Generate document embedding: 100%|██████████| 77/77 [00:14<00:00,  5.42it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [16]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contain key:value pairs mapping 
                                         exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv("""What is the Germany exchange code? Return only the exchange codes as a simple
                                         comma separated value that I can copy.""")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    All exchanges are open during weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
    All exchanges are closed on weekends. Weekends are: Sat, Sun.
    An exchange can only close on a day it was open.
    The fields pre_market and post_market both represent open hours.
    The current date and time is: {datetime.now(est).strftime('%c')}

    When was the US exchange's last operating hours? Provide just the close.
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
  "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
  "SX": "DEUTSCHE BOERSE Stoxx",
  "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
  "DB": "DUBAI FINANCIAL MARKET",
  "NZ": "NEW ZEALAND EXCHANGE LTD",
  "QA": "QATAR EXCHANGE",
  "KS": "KOREA EXCHANGE (STOCK MARKET)",
  "SW": "SWISS EXCHANGE",
  "DU": "BOERSE DUESSELDORF",
  "BC": "BOLSA DE VALORES DE COLOMBIA",
  "KQ": "KOREA EXCHANGE (KOSDAQ)",
  "SN": "SANTIAGO STOCK EXCHANGE",
  "SI": "SINGAPORE EXCHANGE",
  "AD": "ABU DHABI SECURITIES EXCHANGE",
  "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
  "L": "LONDON STOCK EXCHANGE",
  "ME": "MOSCOW EXCHANGE",
  "TO": "TORONTO STOCK EXCHANGE",
  "BD": "BUDAPEST STOCK EXCHANGE",
  "TG": "DEUTSCHE BOERSE TradeGate",
  "US": "US exchanges (NYSE, Nasdaq)",
  "TW": "TAIWAN STOCK EXCHANGE",
  "JK": "INDONESIA STOCK EXCHANGE",
  "SZ": "SHENZHEN STOCK EXCHANGE",
  "VS": "NASDAQ OMX VILNIUS",
  "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
  "DE": "XETRA",
  "PR": "PRAGUE STOCK EXCHANGE",
  "

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [17]:
# Declare callable functions using OpenAPI schema
get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

get_local_datetime_1 = types.FunctionDeclaration(
    name="get_local_datetime_1",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format your final response.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Search for the current market status of global exchanges. Checks whether exchanges are open or 
                   closed. The response is provided in json format. Each response contains the following key-value 
                   pairs:

                   exchange: Exchange code,
                   timezone: Timezone,
                    holiday: Holiday event name, or n/a if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds,
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or n/a if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from now's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The
                                  default value is now's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Search for a daily summary stock ticker candlestick / aggregate bar (OHLC). 
                   Includes open, high, low, and close price. Also includes daily trade volume and pre-market/
                   after-hours trade prices.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is the date 
                                  from calling get_last_market_close. This date can never be more recent than
                                  get_last_market_close."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval (in Eastern Time). Includes open, high, low, and close price. Also includes daily 
                   trade volume and pre-market/after-hours trade prices.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "Specifies the size of the timespan multiplier. The default value is 1."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from now's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The
                                  default value is now's date."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this custom result. The default is 
                                  5000 and the maximum is 50000."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "query", "adjusted", "sort", "limit"]
    },
)

get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Return results published on, before, or after this date in UTC. An example date 
                                  looks like this 2025-04-10T00:24:00Z. The default is to omit this value unless
                                  specified."""
            },
            "order": {
                "type": "string",
                "description": """Must be asc if ascending order, or desc for decending ordering.
                                  When order is omitted default to ascending ordering.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This is allowed to range from 100 to 1000."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "order", "limit", "sort", "query"]
    }
)

get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [18]:
# Implement the callable functions and the function handler

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"], max_sources = 50).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def rag_exchange_codes_1(content):
    response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contaihttps://api.polygon.io/v3/reference/tickers/AAPL?apiKey=4xJe226Z23RZmEc1bN8az1zz4pmNWdOpn key:value pairs 
                                             mapping exchange code to name. Just the dictionary string.
                                             Omit all other information or details. Do not chat or use sentences.""")
    codes = list(ast.literal_eval(response.text.strip("\`")).items())
    return codes

def rag_exchange_code_1(content):
    codes = tool_rag.get_exchanges_csv(f"""What is the {content} exchange code? Return only the exchange codes 
                                           as a list in string form. Just the list string.
                                           Omit all other information or details. Do not chat or use sentences.""").text
    return ast.literal_eval(codes)
    
def rag_last_market_close(content):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    return tool_rag.get_exchanges_csv(
        f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open during weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        All exchanges are closed on weekends. Weekends are: Sat, Sun.
        An exchange can only close on a day it was open.
        The fields pre_market and post_market both represent open hours.
        The current date and time is: {datetime.now(est).strftime('%c')}
    
        When was the US exchange's last operating hours? Provide just the close.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

def get_similarity_score(content):
    similarity = client.models.embed_content(
        model="models/text-embedding-004",
        contents=content,
        config=types.EmbedContentConfig(task_type="semantic_similarity"))
    df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
    score = df @ df.T
    return score.iloc[0].iloc[1]
    
def impl_get_symbol_1(content, by_name: bool = True):
    response = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(response[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}"
        try:
            response = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            matches = []
            max_failed_match = len(response["result"]) if not by_name else 3
            p_desc_match = 0.80
            p_symb_match = 0.95
            if response["count"] > 0:
                for result in tqdm(response["result"], desc="Score similarity to query"):
                    if max_failed_match > 0:
                        desc = [content['q'].upper(), result["description"].split("-", -1)[0]]
                        symb = [content['q'].upper(), result["symbol"]]
                        if by_name and get_similarity_score(desc) > p_desc_match: 
                            matches.append(result["symbol"])
                        elif not by_name and get_similarity_score(symb) > p_symb_match:
                            matches.append(result["description"])
                            max_failed_match = 0
                        else:
                            max_failed_match -= 1
            if len(matches) > 0:
                tool_rag.add_api_document(content["query"], ", ".join(matches), content["q"], "get_symbol_1")
                return ", ".join(matches)
            else:
                return "I don't know."
    else:
        doc = ast.literal_eval(response[0][0])[0]
        return doc["answer"]

def impl_get_name_1(content):
    return impl_get_symbol_1(content, by_name = False)

def impl_get_quote_1(content):
    quotes = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = dict(impl_get_market_status_1(content))["isOpen"]
    if len(quotes[0]) == 0 or isOpen: 
        return get_current_price_1(content)
    else:
        last_close = parse(rag_last_market_close(content)).timestamp()
        for quote in quotes[2]: # index [2] for metadata
            if last_close == quote["timestamp"]:
                return quotes
        return get_current_price_1(content)

def get_current_price_1(content):
    url = f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    # This is a high-demand endpoint. Expect random failure under heavy (free) use.
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0 and response["t"] > 0:
            tool_rag.add_quote_document(content["query"], response, content["symbol"], response["t"], "get_quote_1")
            return list(response.items())
        return "I don't know."

def impl_get_market_status_1(content):
    url = f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}"
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0:
            return list(response.items())
        return "I don't know."

def impl_get_peers_1(content):
    docs = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(docs[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}"
        try:
            peers = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(peers) > 0:
                names = []
                for peer in peers:
                    if peer == content["symbol"]:
                        continue # skip including the query symbol in peers (included in metadata anyway)
                    name_lookup = dict(q=peer, exchange=content["exchange"], query=content["query"])
                    name = impl_get_name_1(name_lookup)
                    if name != "I don't know.":
                        p = {"symbol": peer, "name": name}
                        names.append(p)
                peers = {"symbol": content["symbol"], "peers": names}
                tool_rag.add_peers_document(content["query"], peers, content["symbol"], "get_peers_1", content['grouping'])
                return list(peers.items())
            return "I don't know."
    else:
        peers = ast.literal_eval(docs[0][0])[0]["answer"] # The first document should be most relevant.
        return list(peers.items())

def impl_local_datetime_1(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(datetime.fromtimestamp(timestamp).strftime('%c'))
    return local_t

def impl_get_financials_1(content):
    fins = tool_rag.get_api_documents(content["query"], content["symbol"], "get_financials_1")
    if len(fins[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}"
        try:
            fin = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if not fin:
                return "I don't know."
            tool_rag.add_api_document(content["query"], fin, content["symbol"], "get_financials_1")
            return list(fin.items())
    return fins

def impl_get_news_1(content):
    news = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(news[0]) == 0:
        url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
        try:
            news = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(news) == 0:
                return "I don't know."
            tool_rag.add_api_document(content["query"], news, content["symbol"], "get_news_1")
            return news
    return news

def impl_daily_candle_2(content):
    daily_candle = tool_rag.get_api_documents(content["query"], content["stocksTicker"], "daily_candle_2")
    if len(daily_candle[0]) == 0:
        url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            daily_candle = ast.literal_eval(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if daily_candle["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], daily_candle, content["stocksTicker"], "daily_candle_2")
                return list(daily_candle.items())
            else:
                date = parse(content["date"])
                new_date = None
                if date.weekday() == 4: # index 4 for friday
                    new_date = date - timedelta(days=1)
                elif date.weekday() == 0: # index 0 for monday
                    new_date = date - timedelta(days=3)
                if new_date is None:
                    return "I don't know."
                content["date"] = new_date.strftime("%Y-%m-%d")
                return impl_daily_candle_2(content)
    return daily_candle

def impl_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        request = requests.get(url)
        custom_candle = json.loads(request.text)
    except:
        return f"I don't know. Endpoint returned status {request.status_code}"
    else:
        if custom_candle["status"] in ["OK","DELAYED"]:
            tool_rag.add_api_document(content["query"], custom_candle, content["stocksTicker"], "custom_candle_2")
            return list(custom_candle.items())
        return "I don't know."

def impl_ticker_overview_2(content):
    overview = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(overview[0]) == 0:
        url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            overview = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if overview["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], overview, content["ticker"], "ticker_overview_2")
                return list(overview.items())
            return "I don't know."
    return overview

def impl_trends_1(content):
    trends = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(trends[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
        try:
            trends = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(trends) > 0:
                tool_rag.add_api_document(content["query"], trends, content["symbol"], "trends_1")
                return trends
            return "I don't know."
    return trends

def impl_get_news_2(content):
    news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return "I don't know."
    return news
        
finance_tool = types.Tool(
    function_declarations=[
        get_symbol_1,
        get_name_1,
        get_symbol_quote_1,
        get_market_status_1,
        get_company_peers_1,
        get_local_datetime_1,
        get_last_market_close,
        get_exchange_codes_1,
        get_exchange_code_1,
        get_financials_1,
        get_company_news_1,
        get_daily_candlestick_2,
        get_custom_candlestick_2,
        get_ticker_overview_2,
        get_recommendation_trends_1,
        get_news_with_sentiment_2,
        get_rag_tool_response,
        get_wiki_tool_response,
        get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": impl_get_symbol_1,
    "get_name_1": impl_get_name_1,
    "get_symbol_quote_1": impl_get_quote_1,
    "get_market_status_1": impl_get_market_status_1,
    "get_company_peers_1": impl_get_peers_1,
    "get_local_datetime_1": impl_local_datetime_1,
    "get_last_market_close": rag_last_market_close,
    "get_exchange_codes_1": rag_exchange_codes_1,
    "get_exchange_code_1": rag_exchange_code_1,
    "get_financials_1": impl_get_financials_1,
    "get_company_news_1": impl_get_news_1,
    "get_daily_candlestick_2": impl_daily_candle_2,
    "get_custom_candlestick_2": impl_custom_candle_2,
    "get_ticker_overview_2": impl_ticker_overview_2,
    "get_recommendation_trends_1": impl_trends_1,
    "get_news_with_sentiment_2": impl_get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [19]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: The current date is {datetime.now().strftime('%c')} in Eastern Time.
RULE#2: Always consult your other functions before get_search_tool_response.
RULE#3: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#4: Always consult get_search_tool_response last.
RULE#5: Always respond incorporating as much useful information from function responses.
RULE#6: Always convert timestamps from epoch time using get_local_datetime_1."""

In [20]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [21]:
# Implement the function calling expert.

def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    contents += """
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = client.models.generate_content(
            model=project_model, config=config_fncall, contents=contents
        )
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [23]:
send_message("What is the current price of Amazon stock?")

#### Natural language response

The current price of Amazon (AMZN) is \\$173.18. The price changed by \\$5.86, which is a 3.5023% increase. The high price of the day was \\$176.78, and the low price of the day was \\$169.35. The opening price of the day was \\$169.845, and the previous close price was \\$167.32. The price was last updated on Tue Apr 22 2025 at 20:00:00.


In [24]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Present the data with multiple columns for display in markdown.""")

Generate api embedding: 0it [00:00, ?it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Here is a summary of Amazon's (AMZN) stock information:

**Current Share Price:** As of April 22, 2025, the current price is \\$173.18, which is up \\$5.86 (3.50%) from the previous close.

**Candlestick Data (Past Month):**

| Date               | Open      | High      | Low       | Close     | Volume      |
| :----------------- | :-------- | :-------- | :-------- | :-------- | :---------- |
| April 22, 2025     | \\$169.85   | \\$176.78   | \\$169.35   | \\$173.18   | 56,607,202  |
| April 21, 2025     | \\$169.60   | \\$169.60   | \\$165.29   | \\$167.32   | 48,126,111  |
| April 17, 2025     | \\$176.00   | \\$176.21   | \\$172.00   | \\$172.61   | 44,726,453  |
| April 16, 2025     | \\$176.29   | \\$179.10   | \\$171.41   | \\$174.33   | 51,866,916  |
| April 15, 2025     | \\$181.41   | \\$182.35   | \\$177.93   | \\$179.59   | 43,617,902  |
| April 14, 2025     | \\$186.84   | \\$187.44   | \\$179.23   | \\$182.12   | 48,002,540  |
| April 11, 2025     | \\$179.93   | \\$185.86   | \\$178.00   | \\$184.87   | 50,594,339  |
| April 10, 2025     | \\$185.44   | \\$186.87   | \\$175.85   | \\$181.22   | 68,302,045  |
| April 09, 2025     | \\$172.12   | \\$192.65   | \\$169.93   | \\$191.10   | 116,804,328 |
| April 08, 2025     | \\$185.23   | \\$185.90   | \\$168.57   | \\$170.66   | 87,710,360  |
| April 07, 2025     | \\$162.00   | \\$183.41   | \\$161.38   | \\$175.26   | 109,297,115 |
| April 04, 2025     | \\$167.15   | \\$178.14   | \\$166.00   | \\$171.00   | 123,136,859 |
| April 03, 2025     | \\$183.00   | \\$184.13   | \\$176.92   | \\$178.41   | 95,553,617  |
| April 02, 2025     | \\$187.66   | \\$198.34   | \\$187.66   | \\$196.01   | 53,679,198  |
| April 01, 2025     | \\$187.86   | \\$193.93   | \\$187.20   | \\$192.17   | 41,246,065  |
| March 31, 2025     | \\$188.19   | \\$191.33   | \\$184.40   | \\$190.26   | 63,543,658  |
| March 28, 2025     | \\$198.42   | \\$199.26   | \\$191.88   | \\$192.72   | 52,542,526  |
| March 27, 2025     | \\$200.89   | \\$203.79   | \\$199.28   | \\$201.36   | 27,317,661  |
| March 26, 2025     | \\$205.84   | \\$206.01   | \\$199.93   | \\$201.13   | 32,990,973  |
| March 25, 2025     | \\$203.60   | \\$206.21   | \\$203.22   | \\$205.71   | 31,171,161  |
| March 24, 2025     | \\$200.00   | \\$203.64   | \\$199.95   | \\$203.26   | 41,583,365  |

**Summary:**

Over the past month, Amazon's stock price has shown volatility. The price has decreased from approximately \\$203 to \\$173.18. The volume has also fluctuated, with some days showing significantly higher trading activity.


In [25]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

Generate api embedding: 0it [00:00, ?it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Based on the information gathered, here's a summary of Amazon's (AMZN) current analyst recommendations and sentiment:

**Recommendation Trends:**

*   The analyst recommendation trends for Amazon show a strong bullish sentiment.
*   Over the past four months (January 2025 - April 2025), the number of "Buy" and "Strong Buy" recommendations has consistently outweighed "Hold," "Sell," and "Strong Sell" recommendations.
*   Specifically, in April 2025, there were 50 "Buy" and 23 "Strong Buy" recommendations, compared to 4 "Hold" recommendations, and no "Sell" or "Strong Sell" recommendations.

**Sentiment Analysis:**

*   The news articles related to Amazon display a mixed sentiment.
*   Recent articles from April 2021 discuss Amazon's agreement with Clean Energy Fuels for renewable natural gas, initially causing Clean Energy Fuels' stock to soar. However, subsequent articles pointed out that warrants given to Amazon as part of the deal led to Clean Energy Fuels' stock dropping.
*   Other articles suggest that Amazon, along with Alibaba, are "blue-chip bargains" ahead of earnings.
*   There is also news about Amazon opening a hair salon to trial new technologies.

**Concise Summary:**

Amazon has a predominantly bullish outlook based on analyst recommendations, with a high number of "Buy" and "Strong Buy" ratings. Recent news sentiment is mixed, with positive coverage of new ventures and partnerships offset by concerns about the financial implications of certain deals.


In [27]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 19.80it/s]
Generate api embedding: 0it [00:00, ?it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Google's (Alphabet Inc.) share price and news sentiment over the past month:

**Share Price Summary (GOOGL):**

*   The share price of Alphabet Inc. (GOOGL) has fluctuated over the past month.
*   Starting from approximately \\$167.68 on March 23, 2025, it experienced volatility, reaching a low of \\$145.60 around March 31, 2025, and a high of \\$159.07 around April 18, 2025.
*   The share price closed at \\$151.47 on April 22, 2025.

**News Sentiment Analysis:**

*   The news articles from the past month contain a mix of positive, negative, and neutral sentiments.
*   Many articles focus on Alphabet's earnings, digital ad revenue, and competition with other tech companies.
*   Some articles discuss potential antitrust concerns and regulatory scrutiny.
*   Recent news mentions a battle between Roku and Google over YouTube TV, as well as Google's Q1 earnings exceeding expectations.

**Concise Summary:**

Alphabet Inc.'s (GOOGL) share price has been volatile over the past month, with a mix of positive and negative news sentiment. The company faces both opportunities and challenges, including strong earnings, competition, and regulatory scrutiny.


In [29]:
send_message(
    '''How is the outlook for Apple based on current trends and sentiment.
    Perform the same analysis on Apple's peers. Then compare Apple to it's peers.''')

Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 22.77it/s]
Generate api embedding: 0it [00:00, ?it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 96.77it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 97.53it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 88.50it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 99.17it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 94.84it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 99.98it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 102.44it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score simila

#### Natural language response

Here's a summary of the outlook for Apple and its peers based on analyst recommendation trends:

**Apple (AAPL):**

*   The latest analyst recommendations for April 2025 show a consensus leaning towards a positive outlook:
    *   Strong Buy: 12
    *   Buy: 23
    *   Hold: 12
    *   Sell: 3
    *   Strong Sell: 1

**Peer Analysis:**

Here's a summary of analyst recommendations for Apple's peers (April 2025):

*   **Dell Technologies (DELL):**
    *   Strong Buy: 6
    *   Buy: 20
    *   Hold: 4
    *   Sell: 0
    *   Strong Sell: 0
*   **HP Inc. (HPQ):**
    *   Strong Buy: 2
    *   Buy: 4
    *   Hold: 13
    *   Sell: 1
    *   Strong Sell: 0
*   **Hewlett Packard Enterprise (HPE):**
    *   Strong Buy: 4
    *   Buy: 7
    *   Hold: 8
    *   Sell: 0
    *   Strong Sell: 0
*   **Super Micro Computer Inc. (SMCI):**
    *   Strong Buy: 2
    *   Buy: 8
    *   Hold: 8
    *   Sell: 2
    *   Strong Sell: 0
*   **NetApp Inc. (NTAP):**
    *   Strong Buy: 3
    *   Buy: 8
    *   Hold: 17
    *   Sell: 0
    *   Strong Sell: 0
*   **Pure Storage Inc. (PSTG):**
    *   Strong Buy: 8
    *   Buy: 13
    *   Hold: 6
    *   Sell: 1
    *   Strong Sell: 0
*   **Western Digital Corp. (WDC):**
    *   Strong Buy: 6
    *   Buy: 15
    *   Hold: 8
    *   Sell: 0
    *   Strong Sell: 0
*   **IonQ Inc. (IONQ):**
    *   Strong Buy: 2
    *   Buy: 7
    *   Hold: 2
    *   Sell: 0
    *   Strong Sell: 0

**Comparative Summary:**

*   **Apple** has a generally positive outlook, with a significant number of "Buy" and "Strong Buy" recommendations. However, it also has some "Sell" and "Strong Sell" recommendations, indicating some level of concern among analysts.
*   **Dell Technologies** shows a very strong positive sentiment, with a high number of "Buy" and "Strong Buy" recommendations and no "Sell" recommendations.
*   **HP Inc.** has a more neutral outlook, with most analysts recommending to "Hold" the stock.
*   **Hewlett Packard Enterprise** has a moderately positive outlook, with more "Buy" and "Strong Buy" recommendations than "Hold."
*   **Super Micro Computer Inc.** has a mixed outlook, with a significant number of "Hold" recommendations and some "Sell" recommendations.
*   **NetApp Inc.** has a moderately positive outlook, with more "Buy" and "Strong Buy" recommendations than "Hold."
*   **Pure Storage Inc.** has a strong positive outlook, with a high number of "Buy" and "Strong Buy" recommendations.
*   **Western Digital Corp.** has a strong positive outlook, with a high number of "Buy" and "Strong Buy" recommendations and no "Sell" recommendations.
*   **IonQ Inc.** has a positive outlook, with more "Buy" and "Strong Buy" recommendations than "Hold."

In summary, Apple's outlook is positive, but some of its peers, like Dell Technologies, Western Digital Corp, and Pure Storage Inc., have even stronger positive sentiment from analysts. HP Inc. has a more neutral outlook compared to Apple and its other peers.


In [28]:
send_message('''Tell me the recent news about Apple over the past two month.''')

Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

Over the past two months, Apple has been active with several releases, updates, and other news.

**Releases and Updates:**

*   Apple has released the second public betas of iOS 18.5 and macOS Sequoia 15.5, allowing public testing.
*   Apple Intelligence features are now available in India and expanding to more languages and regions. These features are also available on Apple Vision Pro with visionOS 2.4.
*   A new Macbook Air with the M4 chip and a sky blue color has been introduced.
*   The Ipad Air has been updated with a powerful M3 chip and a new Magic Keyboard.
*   New games have been launched on Apple Arcade, including Katamari and Space Invaders.

**Products & Rumors:**

*   There are rumors about an all-new super-thin Iphone called the "Iphone 17 Air."
*   The Iphone 17 Pro might come in a Sky Blue color.

**Other News:**

*   Apple is revamping the Siri management team to improve the development of Apple Intelligence features.
*   Apple has surpassed a 60% reduction in global greenhouse gas emissions.
*   Apple's Worldwide Developers Conference will return the week of June 9.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>